## Intersubject Correlation Analysis (ISC)

The BOLD signal contains noise. Here, we calculate correlations between subjects to reduce noise and estimate task-relevant signals.

---

1. Load data in notebook
2. Brief fMRI data exploration
3. Do ISC analysis; see (Chen et al., 2017) and [Brainiak ISC tutorial](https://brainiak.org/tutorials/10-isc/)
    - (A) On the toy datasets from Brainiak
    - (B) On our own data
    
[Brainiak ISC analyasis documentation](https://brainiak.org/docs/brainiak.html#module-brainiak.isc)

[Brainiak specific examples](https://github.com/brainiak/brainiak/tree/master/examples)

In [1]:
# TODO

# ISC on toy dataset ('Pieman')

### Workflow

Always perform the following steps

1. Prepare the data (define data path etc)
2. Compute ISC
3. Permutation test ISC
4. Compute IFSC
5. Cluster IFSC results
6. Perform IFSC permutation

In [2]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 
import glob
import time
from copy import deepcopy
import numpy as np
import pandas as pd
import os

from nilearn import datasets
from nilearn import surface
from nilearn import plotting
from nilearn.input_data import NiftiMasker, NiftiLabelsMasker
import nibabel as nib

import brainiak
from brainiak import image, io
#from brainiak.isc import isc, isfc, permutation_isc # getting error
import matplotlib.pyplot as plt
import seaborn as sns 

%autosave 5
%matplotlib inline
sns.set(style = 'white', context='talk', font_scale=1, rc={"lines.linewidth": 2})

Autosaving every 5 seconds


## 1.1 Data prep

-  All subjects must be in the same anatomical space for analysis.
- create a whole-brain mask
- outcome of this is an array of anatomically-aligned and temporally-aligned brain data.

In [8]:
# define data path and check if dir is valid
pieman2_dir = '/Users/Daphne/Desktop/Pieman2' # local directory
os.path.exists(pieman2_dir)

False

In [4]:
dir_mask = os.path.join(pieman2_dir, 'masks/')
mask_name = os.path.join(dir_mask, 'avg152T1_gray_3mm.nii.gz')
all_task_names = ['word', 'intact1']
all_task_des = ['word level scramble', 'intact story']
n_subjs_total = 10
group_assignment_dict = {task_name: i for i, task_name in enumerate(all_task_names)}

# Where do you want to store the data
dir_out = os.getcwd() + str('/pieman2_data')
if not os.path.exists(dir_out): # make data folder within current working directory
    os.makedirs(dir_out)
    print('Dir %s created ' % dir_out)

## Helper functions

In [5]:
# Reduce the number of subjects per condition to make this notebook faster 
upper_limit_n_subjs = 10

def get_file_names(data_dir_, task_name_, verbose = False):
    """
    Get all the participant file names
    
    Parameters
    ----------
    data_dir_ [str]: the data root dir
    task_name_ [str]: the name of the task 
    
    Return
    ----------
    fnames_ [list]: file names for all subjs
    """
    c_ = 0 
    fnames_ = []
    # Collect all file names 
    for subj in range(1, n_subjs_total): 
        fname = os.path.join(
            data_dir_, 'sub-%.3d/func/sub-%.3d-task-%s.nii.gz' % (subj, subj, task_name_))
        
        # If the file exists
        if os.path.exists(fname):
            
            # Add to the list of file names 
            fnames_.append(fname)
            if verbose: 
                print(fname)
            c_+= 1
            if c_ >= upper_limit_n_subjs: 
                break
    return fnames_

In [6]:
"""load brain template"""

# Load the brain mask
brain_mask = io.load_boolean_mask(mask_name)

# Get the list of nonzero voxel coordinates
coords = np.where(brain_mask)

# Load the brain nii image
brain_nii = nib.load(mask_name)

In [7]:
"""load bold data"""

# load the functional data 
fnames = {}
images = {}
masked_images = {}
bold = {}
group_assignment = []
n_subjs = {}

for task_name in all_task_names: 
    fnames[task_name] = get_file_names(pieman2_dir, task_name)
    images[task_name] = io.load_images(fnames[task_name]) 
    masked_images[task_name] = image.mask_images(images[task_name], brain_mask) 
    # Concatenate all of the masked images across participants  
    bold[task_name] = image.MaskedMultiSubjectData.from_masked_images(
        masked_images[task_name], len(fnames[task_name])
    )
    # Convert nans into zeros
    bold[task_name][np.isnan(bold[task_name])] = 0
    # compute the group assignment label 
    n_subjs_this_task = np.shape(bold[task_name])[-1]
    group_assignment += list(
        np.repeat(group_assignment_dict[task_name], n_subjs_this_task)
    )
    n_subjs[task_name] = np.shape(bold[task_name])[-1]
    print('Data loaded: {} \t shape: {}' .format(task_name, np.shape(bold[task_name])))

StopIteration: 

## Let's look at that data

### TODO: exercises 

Brain template
- Report the shape of brain_nii, brain_mask
- Visualize brain_nii and brain_mask by plotting the 30th slice along the Z dimension.
- Describe what coords refers to
- Visualize coords with a 3d plot. For this, only plot every 10th point, otherwise the plot will be slow to load.

Brain data
- Inspect the shape of bold. How many subjects do we have for each task condition? 
- Do different subjects have the same number of TRs/voxels?

## Brain template

In [ ]:
brain_nii.shape

In [ ]:
brain_mask.shape

In [ ]:
1*brain_mask # brain mask is a binary matrix that selects the ROI's 

In [ ]:
coords # coordinates

In [ ]:
from mpl_toolkits import mplot3d

fig = plt.figure()
ax = plt.axes(projection="3d")
ax.scatter3D(coords[0], coords[1], coords[2]);

In [ ]:
slice_coords = [(x_coord, y_coord, z_coord) for x_coord in x for y_coord in y for z_coord in [10]]
fig = plt.figure()
ax = plt.axes(projection="3d")
ax.scatter3D(slice_coords[0], slice_coords[1], slice_coords[2]);

In [ ]:
df_coords = pd.DataFrame([x, y, z])[:,:,30]
plt.scatter(df_coords)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection="3d")
ax.scatter3D(coords[0], coords[1], 10);

## Bold data

In [47]:
bold['intact1'].shape # voxels, ... , participants

(98508, 300, 8)

## 1.2 Compute ISC

AttributeError: module 'brainiak' has no attribute 'isc'